In [ ]:
# pip install -U pymoo

In [1]:
import numpy as np
import pandas as pd
import json

from data_preprocess import *
from run_ga import *

In [2]:
tet_path = "./data/tet.json"
fdr_path = "./data/all_class_mutation/Chart-1_fdr.json"

with open(tet_path, 'r') as f:
    tet_data = json.load(f)


with open(fdr_path, 'r') as f:
    fdr_data = json.load(f)

chart_1_tet = tet_data['Chart-1']
chart_1_fdr = dict()

for key, value in fdr_data['Chart_1'].items():
    chart_1_fdr[key] = value['mutation-score']


chart_1_tet = sorted(chart_1_tet.items())
chart_1_fdr = sorted(chart_1_fdr.items())

In [6]:
tests = [i[0] for i in chart_1_tet]
execution_times = np.array([i[1] for i in chart_1_tet])
fault_detections = np.array([i[1] for i in chart_1_fdr])

test_cases = np.column_stack((execution_times, fault_detections))
adequacy_scores = get_adequacy_scores(fault_detections)

In [7]:
run_nsga(test_cases)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      5 |             - |             -
     2 |      110 |      9 |  0.4984229818 |         ideal
     3 |      120 |     10 |  0.3200602794 |         ideal
     4 |      130 |     10 |  0.000000E+00 |             f
     5 |      140 |     11 |  0.0455323565 |             f
     6 |      150 |     11 |  0.1522978152 |         ideal
     7 |      160 |     11 |  0.0758169737 |         ideal
     8 |      170 |     13 |  0.0841835320 |         ideal
     9 |      180 |     12 |  0.1148031805 |         ideal
    10 |      190 |     13 |  0.0081002525 |             f
    11 |      200 |     14 |  0.0053821839 |         ideal
    12 |      210 |     14 |  0.0060300470 |             f
    13 |      220 |     16 |  0.1091383528 |         ideal
    14 |      230 |     17 |  0.0050823120 |             f
    15 |      240 |     20 |  0.0612113665 |         ideal
    16 |      250 |     22 |  0.0048433899 |            

In [8]:
run_nsga_with_adequecy(test_cases, adequacy_scores)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      7 |             - |             -
     2 |      110 |     11 |  0.6416173680 |         ideal
     3 |      120 |     13 |  0.0383951475 |             f
     4 |      130 |     11 |  0.2702763241 |         ideal
     5 |      140 |     14 |  0.1812027944 |         ideal
     6 |      150 |     15 |  0.0262905727 |             f
     7 |      160 |     16 |  0.0125466551 |             f
     8 |      170 |     16 |  0.1046471703 |         ideal
     9 |      180 |     16 |  0.1443406951 |         ideal
    10 |      190 |     18 |  0.0099383159 |             f
    11 |      200 |     16 |  0.0116550545 |             f
    12 |      210 |     15 |  0.0107408671 |         ideal
    13 |      220 |     17 |  0.0769150016 |         ideal
    14 |      230 |     19 |  0.0080310763 |             f
    15 |      240 |     22 |  0.0020608152 |             f
    16 |      250 |     26 |  0.0468720469 |         ide